# Scrappe Australie
site :https://www.gumtree.com.au/s-cars-vans-utes/c18320?view=gallery

# Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.firefox.options import Options
import pandas as pd 
import numpy as np 

import random
from selenium.webdriver.common.action_chains import ActionChains
from itertools import combinations
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from fake_useragent import UserAgent

## Fonctions Annexe

In [2]:
def click_button(driver, xpath, timeout=10, max_retries=3):
    for retry in range(max_retries):
        try:
            bouton = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )

            WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((By.XPATH, xpath))
            )

            # Get the initial class name before clicking
            initial_class_name = bouton.get_attribute("class")

            bouton.click()

            # Wait for the class change to occur
            WebDriverWait(driver, timeout).until(
                lambda driver: initial_class_name != bouton.get_attribute("class")
            )

            print("Le bouton a été cliqué avec succès.")
            return True

        except TimeoutException:
            print(f"Timeout de {timeout} secondes atteint. Réessai {retry + 1}/{max_retries}.")

        except StaleElementReferenceException:
            print(f"Stale Element Reference Exception. Réessai {retry + 1}/{max_retries}.")

    print(f"Impossible de trouver et cliquer sur le bouton après {max_retries} tentatives.")
    return False

In [3]:
def scrape_content(driver, xpath, content_type='text', max_retries=3, timeout=8):
    for retry in range(max_retries):
        try:
            # Trouver tous les éléments correspondant à l'XPath fourni
            elements = WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.XPATH, xpath))
            )

            # Extraire le contenu spécifié pour chaque élément
            if content_type == 'text':
                result = [element.text for element in elements]
            elif content_type == 'href':
                result = [element.get_attribute('href') for element in elements]
            else:
                raise ValueError("Type de contenu non pris en charge. Utilisez 'text' ou 'href'.")

            return result

        except TimeoutException:
            print(f"Timeout de {timeout} secondes atteint. Réessai {retry + 1}/{max_retries}.")

        except NoSuchElementException:
            print(f"Élément non trouvé. Vérifiez votre XPath. Réessai {retry + 1}/{max_retries}.")

        except StaleElementReferenceException:
            print(f"Stale Element Reference Exception. L'élément n'est plus attaché au DOM. Réessai {retry + 1}/{max_retries}.")

        except Exception as e:
            print(f"Une erreur s'est produite : {str(e)}")

    print(f"Impossible de trouver et extraire le contenu après {max_retries} tentatives.")
    return []


In [4]:
def action_aleatoire(driver):
    actions_possibles = ["scroll_full", "scroll_half", "move_to_element"]
    action_choisie = random.choice(actions_possibles)

    # Enregistrer la position actuelle de la page
    current_scroll_position = driver.execute_script("return window.pageYOffset;")

    if action_choisie == "scroll_full":
        # Action aléatoire : Faites défiler la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    elif action_choisie == "scroll_half":
        # Action aléatoire : Faites défiler la moitié de la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
    elif action_choisie == "move_to_element":
        # Action aléatoire : Bougez la souris vers un élément aléatoire (par exemple, div de class PriceInformation_classifiedPrice__b-Jae)
        wait = WebDriverWait(driver, 10)
        element_to_move = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'center')))
        driver.execute_script("arguments[0].scrollIntoView();", element_to_move)

    # Revenir à la position enregistrée
    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

In [5]:
def initialize_driver():
    ua = UserAgent()
    user_agent = ua.random

    options = Options()
    options.add_argument('--headless')
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Firefox(options=options)
    return driver

In [6]:
def wait_for_class_change(driver, xpath, old_class_name, timeout=10):
    def class_has_changed(driver):
        element = driver.find_element(By.XPATH, xpath)
        new_class_name = element.get_attribute("class")
        return old_class_name != new_class_name

    try:
        WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        WebDriverWait(driver, timeout).until(class_has_changed)
        return True
    except TimeoutException:
        return False


In [7]:
import os
import csv

def write_to_csv(csv_file_path, marque_modele, prix, contenu_button_1, contenu_button_2, contenu_button_3):
    # Check if the CSV file is empty
    file_exists = os.path.isfile(csv_file_path) and os.path.getsize(csv_file_path) > 0

    # Open the CSV file in append mode
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)

        # If the file is empty, write column names
        if not file_exists:
            column_names = ['Marque_Modele', 'Prix', 'Contenu_Button_1', 'Contenu_Button_2', 'Contenu_Button_3']
            csv_writer.writerow(column_names)

        # Write data for the current entry
        data = [marque_modele, prix, contenu_button_1, contenu_button_2, contenu_button_3]
        csv_writer.writerow(data)
        print("+1")


## lien_ xpath utilisé

lien pour touts les produits :
    dans les div de class user-ad-collection-new-design__wrapper--square := a de class user-ad-square-new-design link link--base-color-inherit link--hover-color-none link--no-underline := on recupe les hrefs sous forme de liste
    
Dans fiche prduit:
    Marque + modele : dans h1 de class vip-ad-title__header = recup texte
    prix : span de class nuser-ad-price__price = on recupe le texte
    
    dans div class vip-ad-attributes__list ( caracteristiques) = les li de class vip-ad-attributes__item = on recupere tout les textes
    
puis cliquer sur le boutton pour  specif

button de class class="Button__button--2NsdC Button__buttonBasic--3CSBx undefined Button__buttonBase--3YR6h Button-main__button--3RLVh undefined tab-set__header-button tab-set__header-button--selected" avec event 

dans chaque li du ul de class accordion vip-ad-car-specs. on recupere les attribute text

puis bouttons de class Button__button--2NsdC Button__buttonBasic--3CSBx undefined Button__buttonBase--3YR6h Button-main__button--3RLVh undefined tab-set__header-button tab-set__header-button--selected avec event
    on recupere les features dans tout les li =: les li se trouvents dans ul de class vip-ad__features-list = on .Text
    

In [8]:
# Initialize the driver
driver = initialize_driver()

# Load the page
driver.get("https://www.gumtree.com.au/s-cars-vans-utes/c18320?view=gallery")

# Define the XPath for the elements
xpath = ".//div[@class='user-ad-collection-new-design__wrapper--square']//a[@class='user-ad-square-new-design link link--base-color-inherit link--hover-color-none link--no-underline']"

try:
    # Wait for the elements to be present
    elements_present = EC.presence_of_all_elements_located((By.XPATH, xpath))
    WebDriverWait(driver, 5).until(elements_present)

    # Call the scrape_content function to retrieve hrefs
    all_pages_href = scrape_content(driver, xpath, content_type='href')
    print(all_pages_href)
    try:
        for car_page in all_pages_href:
            # Open the link
            driver.get(car_page)
            print(car_page)
            # XPath for Marque + modele
            marque_modele_xpath = ".//h1[@class='vip-ad-title__header']"
            marque_modele = scrape_content(driver, marque_modele_xpath, content_type='text')

            # XPath for prix
            prix_xpath = ".//span[@class='user-ad-price__price']"
            prix = scrape_content(driver, prix_xpath, content_type='text')

            

            # Iterate through each button
            # XPath pour les boutons dans panel-header
            button_xpath = ".//div[@class='panel-header panel-header--srp tab-set__header']//button"

            # XPath pour le contenu de la div tab_set
            tab_set_content_xpath = ".//div[@class='tab-set__content']"

            # Récupérer tous les boutons
            buttons = driver.find_elements(By.XPATH, button_xpath)

            # Pour chaque bouton
            for i, button in enumerate(buttons, start=1):
                # Cliquez sur le bouton
                button.click()

                try:
                    # Attendre que la classe --selected apparaisse dans le bouton
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"{button_xpath}[{i}][contains(@class, '--selected')]"))
                    )

                    # Scrapper le contenu de la div tab_set
                    tab_set_content = scrape_content(driver, tab_set_content_xpath, content_type='text')

                    # Afficher le contenu
                    print(f"Contenu après avoir cliqué sur le bouton {i}: {tab_set_content}")

                    # Créer un dictionnaire pour stocker les valeurs de chaque bouton
                    button_content_dict = {
                        f"contenu_button_{i}": tab_set_content
                    }

                    # Ajouter les valeurs scrappées à la variable créée dynamiquement
                    locals().update(button_content_dict)

                    # Écrire les données dans le fichier CSV à chaque itération
                    write_to_csv('car_data.csv', marque_modele, prix, **button_content_dict)

                except TimeoutException:
                    print(f"Timeout après avoir cliqué sur le bouton {i}")
                if i == 4:
                    break

    except Exception as e:
        print(f"An error occurred: {str(e)}")

except TimeoutException:
    print("Timed out waiting for page to load")

finally:
    # Close the driver when done
    driver.quit()

['https://www.gumtree.com.au/s-ad/warwick-farm/cars-vans-utes/2020-volvo-xc60-t5-momentum-awd-/1300071129', 'https://www.gumtree.com.au/s-ad/warwick-farm/cars-vans-utes/2013-volvo-xc60-t5-teknik/1300075229', 'https://www.gumtree.com.au/s-ad/parap/cars-vans-utes/2021-isuzu-d-max-white-dual-cab/1304535256', 'https://www.gumtree.com.au/s-ad/alexandria/cars-vans-utes/volkswagen-amarok-tdi580s-4motion-2020/1295673332', 'https://www.gumtree.com.au/s-ad/pinkenba/cars-vans-utes/2020-ldv-t60-sk8c-trailrider-2-blue-6-speed-sports-automatic-utility/1316444531', 'https://www.gumtree.com.au/s-ad/aspley/cars-vans-utes/2020-ford-focus-sa-2020-25my-st-red-7-speed-automatic-hatchback/1320433443', 'https://www.gumtree.com.au/s-ad/west-footscray/cars-vans-utes/2012-volkswagen-amarok-2h-my13-tdi420-4motion-perm-trendline-grey-8-speed-automatic-utility/1318975666', 'https://www.gumtree.com.au/s-ad/morley/cars-vans-utes/2021-mg-hs-sas23-my21-core-dct-fwd-white-7-speed-sports-automatic-dual-clutch-wagon/1319

try button# XPath for the buttons in the panel-header
            button_xpath = ".//div[@class='panel-header panel-header--srp tab-set__header']//button[@class='Button__button--2NsdC Button__buttonBasic--3CSBx undefined Button__buttonBase--3YR6h Button-main__button--3RLVh undefined tab-set__header-button']"

            # Iterate through each button
            for i in range(1, 4):  # Assuming there are three buttons, adjust as needed
                # Construct the XPath for the current button
                current_button_xpath = f"({button_xpath})[{i}]"

                click_success = click_button(driver, current_button_xpath)

                # Check if the button click was successful
                if click_success:
                    # Get the initial class name before clicking
                    bouton = driver.find_element(By.XPATH, current_button_xpath)
                    initial_class_name = bouton.get_attribute("class")

                    # Wait for the class change to occur
                    wait_success = wait_for_class_change(driver, current_button_xpath, initial_class_name)

                    if wait_success:
                        # Scrape content after clicking the button
                        scraped_content_xpath = ".//ul[@class='vip-ad-attributes__list']//li[@class='vip-ad-attributes__item']"
                        scraped_content = scrape_content(driver, scraped_content_xpath, content_type='text')

                        # Print the extracted information
                        print(f"Content after clicking button {i}: {scraped_content}")
                    else:
                        print(f"Timeout waiting for class change after clicking button {i}")